### Setting the feelpp environment


In [ ]:
import torch
import numpy as np
import sympy as sp
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from feelpp.scimba.scimba_pinns import Run_laplacian2D, Poisson_2D, PoissonDisk2D
from scimba.equations import domain


xdomain = domain.SpaceDomain(2, domain.SquareDomain(2, [[0.0, 1.0], [0.0, 1.0]]))
pde = Poisson_2D(xdomain)
network, pde = Run_laplacian2D(pde)
# Extract solution function u
u_scimba = network.forward

# Step 1: Generate a grid of (x, y) points and evaluate the network

# Define a grid of points
x_vals = np.linspace(0, 1, 50)
y_vals = np.linspace(0, 1, 50)
xx, yy = np.meshgrid(x_vals, y_vals)
grid_points = np.vstack([xx.ravel(), yy.ravel()]).T

# Convert to tensor and evaluate the neural network
coordinates_tensor = torch.tensor(grid_points, dtype=torch.double)
mu_value = 1
mu = torch.full((coordinates_tensor.size(0), 1), mu_value, dtype=torch.double)

with torch.no_grad():  # Ensure no gradient computations
    solution_tensor = u_scimba(coordinates_tensor, mu).numpy()

# Step 2: Perform symbolic regression using polynomial regression

# Prepare the polynomial features
poly = PolynomialFeatures(degree=3)  # Degree 3 for cubic polynomials
X_poly = poly.fit_transform(grid_points)

# Fit a linear regression model to the polynomial features
model = LinearRegression()
model.fit(X_poly, solution_tensor.ravel())

# Extract the polynomial coefficients
coefficients = model.coef_
intercept = model.intercept_

# Step 3: Convert the fitted polynomial to a symbolic expression

# Define symbolic variables
x, y = sp.symbols('x y')
poly_feature_names = poly.get_feature_names(input_features=['x', 'y'])

symbolic_expr = sp.sympify(intercept)
for coef, feature in zip(coefficients, poly_feature_names):
    if coef != 0:
        coef = round(coef, 4)
        terms = feature.split()
        expr_terms = []
        for term in terms:
            if term == 'x':
                expr_terms.append(x)
            elif term == 'y':
                expr_terms.append(y)
            else:
                expr_terms.append(sp.symbols(term))
        symbolic_expr += coef * sp.Mul(*expr_terms)

# Print the symbolic expression
expr_str = str(symbolic_expr)
expr_str = expr_str.replace('x^2', 'x*x')
expr_str = expr_str.replace('y^2', 'y*y')
expr_str = expr_str.replace('x^3', 'x*x*x')
expr_str = expr_str.replace('y^3', 'y*y*y')

print(expr_str)



In [ ]:
from tools.GmeshRead import mesh2d
import pyvista as pv
import pandas as pd
import torch

# File path to the .case file
file_path = 'feelppdb/feelpp_cfpde/np_1/cfpdes-2d-p1.exports/Export.case'

# Read the .case file using PyVista
data = pv.read(file_path)

# Iterate over each block in the dataset to find coordinates
coordinates = None
for i, block in enumerate(data):
    if block is None:
        continue
    # Extract the mesh points (coordinates)
    coordinates = block.points
    solution = 'cfpdes.poisson.u'
    solution_expression = block.point_data[solution]

    df = pd.DataFrame(block.point_data)
    print(df.head())
# Considering only 2d problems:
num_features = coordinates.shape[1]
if num_features > 2:
    coordinates = coordinates[:, :2]

print(f"Number of points: {len(coordinates)}")          
print("\nNodes:"  , coordinates)

mesh = "feelppdb/feelpp_cfpde/np_1/omega-2.msh"
my_mesh = mesh2d(mesh)
my_mesh.read_mesh()
print("Number of nodes:", my_mesh.Nnodes)
print("Nodes coordinates:", my_mesh.Nodes)
for i in range(517):
    print(f"Node {i} = {coordinates[i], my_mesh.Nodes[i]}")

"""
print('difference = ', coordinates - my_mesh.Nodes)
feel_solution = block.point_data['cfpdes.poisson.u']
print("\nFeel++ solution 'cfpdes.poisson.u':")
print(feel_solution) 
"""

In [ ]:
import sys
import feelpp,core as fppc
import feelpp.toolboxes.core as tb
from feelpp.scimba.Poisson import Poisson, runLaplacianPk, runConvergenceAnalysis, plot_convergence, custom_cmap

sys.argv = ["feelpp_app"]
e = feelpp.Environment(sys.argv,
                       opts=tb.toolboxes_options("coefficient-form-pdes", "cfpdes"),
                       config=feelpp.localRepository('feelpp_cfpde'))

# ------------------------------------------------------------------------- #
# Poisson problem
# - div (diff * grad (u)) = f    in Omega
#                     u   = g    in Gamma_D
# Omega = domain, either cube or ball
# Approx = lagrange Pk of order order
# mesh of size h

P = Poisson(dim = 2)
P(solver ='scimba', u_exact = expr_str)


### Examples with different parameters

In [ ]:

# for square domain

u_exact = 'sin(2*pi*x) * sin(2*pi*y)'
rhs = '8*pi*pi*sin(2*pi*x) * sin(2*pi*y)'

P(rhs=rhs, g='0', solver='feelpp', u_exact = u_exact)
P(rhs=rhs, g='0', solver ='scimba', u_exact = u_exact)


In [ ]:

u_exact = 'y + (x*(1-x) + y*(1-y)/4) '
P(rhs='5/2', g='y', solver='feelpp', u_exact = u_exact)
P(rhs='5/2', g='y', solver ='scimba', u_exact = u_exact)


In [ ]:

u_exact = '-y*y/2 - x*y*y*y/2 + y*y*y*y/4'
P( rhs='-1.0-3*y*x+y*y', g='-y*y/2 - x*y*y*y/2 + y*y*y*y/4', solver='feelpp', u_exact = u_exact)
P( rhs='-1.0-3*y*x+y*y', g='-y*y/2 - x*y*y*y/2 + y*y*y*y/4', solver ='scimba', u_exact = u_exact)

### Computing errors

In [ ]:
# Collect data to compute errors
u_exact = 'sin(2*pi*x) * sin(2*pi*y)'
grad_u_exact = '{2*pi*cos(2*pi*x) * sin(2*pi*y), 2*pi*sin(2*pi*x) * cos(2*pi*y)}'
rhs = '8*pi*pi*sin(2*pi*x) * sin(2*pi*y)'

h= [0.1, 0.05, 0.025, 0.0125]
measures = []
"""
for i in h:
  P(h=i, rhs=rhs, g='0', plot = None, u_exact = u_exact)
  measures.append(P.measures)

print(measures)
"""
P(h=h[0], rhs=rhs, g='0', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)


P(h=h[1], rhs=rhs, g='0', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)

P(h=h[2], rhs=rhs, g='0', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)

P(h=h[3], rhs=rhs, g='0', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)

print('\n', measures)


# Plotting the error convergence rates
poisson_json = lambda order,dim=2,name="u": P.model
df= runConvergenceAnalysis( P, json=poisson_json, measures=measures, dim=2,verbose=True)
print('measures = ', measures)
fig= plot_convergence(P, df,dim=2)
fig.show()


In [ ]:
# Collect data to compute errors
u_exact = 'y + (x*(1-x) + y*(1-y)/4) '
grad_u_exact = '{1 - 2*x, (5 - 2*y)/4}'
rhs='5/2'

h= [0.1, 0.05, 0.025, 0.0125]
measures = []
"""
for i in h:
  P(h=i, rhs=rhs, g='y*y*y * (1 - y) - 2 * y*y * ((y - 1) * x * (1 - x)) + 6 * y * (1 - y)', plot = None, u_exact = u_exact)
  measures.append(P.measures)

print(measures)
"""
P(h=h[0], rhs=rhs, g='y*y*y * (1 - y) - 2 * y*y * ((y - 1) * x * (1 - x)) + 6 * y * (1 - y)', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)


P(h=h[1], rhs=rhs, g='y*y*y * (1 - y) - 2 * y*y * ((y - 1) * x * (1 - x)) + 6 * y * (1 - y)', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)

P(h=h[2], rhs=rhs, g='y*y*y * (1 - y) - 2 * y*y * ((y - 1) * x * (1 - x)) + 6 * y * (1 - y)', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)

P(h=h[3], rhs=rhs, g='y*y*y * (1 - y) - 2 * y*y * ((y - 1) * x * (1 - x)) + 6 * y * (1 - y)', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)

print('\n', measures)

# Plotting the error convergence rates

poisson_json = lambda order,dim=2,name="u": P.model
df= runConvergenceAnalysis( P, json=poisson_json, measures=measures, dim=2,verbose=True)
print('measures = ', measures)
fig= plot_convergence(P, df,dim=2)
fig.show()


In [ ]:
# Collect data to compute errors
u_exact = 'y*y*y * (1 - y) - 2 * y*y * ((y - 1) * x * (1 - x)) + 6 * y * (1 - y)'
grad_u_exact = '{-2*x*y*y + 6*x*y - 2*x*y*y*y + 6*x*y*y - 2*x*x*y*y + 6*x*x*y , -2*x*x*y + 6*x*x*y - 2*x*x*y*y + 6*x*x*y*y - 2*x*y*y + 6*x*y*y}'
rhs='-2 * (-6 + x * (2 - 6 * y) + 3 * y - 8 * y*y + 2 * y*y*y + x*x * (-2 + 6 * y))'

h= [0.1, 0.05, 0.025, 0.0125]
measures = []
"""
for i in h:
  P(h=i, rhs=rhs, g='y*y*y * (1 - y) - 2 * y*y * ((y - 1) * x * (1 - x)) + 6 * y * (1 - y)', plot = None, u_exact = u_exact)
  measures.append(P.measures)

print(measures)
"""
P(h=h[0], rhs=rhs, g='y*y*y * (1 - y) - 2 * y*y * ((y - 1) * x * (1 - x)) + 6 * y * (1 - y)', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)


P(h=h[1], rhs=rhs, g='y*y*y * (1 - y) - 2 * y*y * ((y - 1) * x * (1 - x)) + 6 * y * (1 - y)', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)

P(h=h[2], rhs=rhs, g='y*y*y * (1 - y) - 2 * y*y * ((y - 1) * x * (1 - x)) + 6 * y * (1 - y)', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)

P(h=h[3], rhs=rhs, g='y*y*y * (1 - y) - 2 * y*y * ((y - 1) * x * (1 - x)) + 6 * y * (1 - y)', plot = None, u_exact = u_exact, grad_u_exact=grad_u_exact)
print('measures = ',P.measures)
measures.append(P.measures)

print('\n', measures)

# Plotting the error convergence rates

poisson_json = lambda order,dim=2,name="u": P.model
df= runConvergenceAnalysis( P, json=poisson_json, measures=measures, dim=2,verbose=True)
print('measures = ', measures)
fig= plot_convergence(P, df,dim=2)
fig.show()

In [ ]:

"""
# # 2D with varying anisotropy
P = Poisson(dim = 2)
u_exact = 'x*x/(1+x) + y*y/(1+y)'
P(rhs='4', diff='{1+x,0,0,1+y}', g='x*x/(1+x) + y*y/(1+y)', solver='feelpp', u_exact = u_exact)
P(rhs='4', diff='{1+x,0,0,1+y}', g='x*x/(1+x) + y*y/(1+y)', solver='scimba', u_exact = u_exact)

u_exact = 'x*x + y*y'
P(rhs='4', diff='{x,y,-y,x+y}', g='x*x + y*y',  solver='feelp', u_exact = u_exact)

poisson_json = lambda order,dim=2,name="u": P.model
print(poisson_json(dim=2,order=1))
df= runConvergenceAnalysis( P, json=poisson_json, measures=measures, dim=2,verbose=True)
print('measures = ', measures)
fig= plot_convergence(P, df,dim=2)
fig.show()
"""
